In [ ]:
#necessary imports for running a gradient boosting classifier 
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc


In [ ]:
#import matrix
matrix = pd.read_csv('/home/nhen/gwas_005_matrix_chr1_22_LDclumped_r2.1.csv', sep=',', header=None, low_memory=False)

In [ ]:
#reconstruct matrices into formats that are parsible 
#necessary format for train_test_split
matrix_re = matrix.transpose()
new_df = matrix_re.dropna()
matrix_re = matrix.transpose()
matrix_noNA = matrix_re.dropna()
data_labeled = matrix_noNA.transpose()
data = data_labeled.drop(data_labeled.columns[[0]], axis=1)


In [ ]:
#pull out labels from full matrix and store y
y = data.iloc[3] #builds object of only phenotype
y = pd.DataFrame(y) #turns into dataframe
y = y.astype(int)
#y = y.subtract(1)

In [ ]:
#pull out allele frequencies from full matrix and store X
X = data.iloc[17:] #builds df of chrs on (post population info)
X = X.transpose()

In [ ]:
#split X and y into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [ ]:
#set classifier and run Gradient Boosting program from scikit learn
#parameters n_estimators, learning_rate and random state were changed and metrics checked before finding and choosing highest accuracy and AUC
#fit to test and train data
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=1).fit(X_train, y_train.values.ravel())

In [ ]:
#mean accuracy on x test and labels
clf.score(X_test, y_test)

In [ ]:
#mean accuracy on training data
clf.score(X_train, y_train)

In [ ]:
#Predict confidence scores for samples. The confidence score for a sample is proportional to the signed distance of that sample to the hyperplane.
print(clf.decision_function(X_test))

In [ ]:
#create validation split
#changed random_state to see optimal performance
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train.values.ravel(), random_state=0)

In [ ]:
#create confusion matrix and report metrics precision, recall, f1
gb = GradientBoostingClassifier(n_estimators=100, learning_rate = 0.5, max_depth = 2, random_state = 1)
gb.fit(X_train, y_train)
predictions = gb.predict(X_validation)

print("Confusion Matrix:")
print(confusion_matrix(y_validation, predictions))
print()
print("Classification Report")
print(classification_report(y_validation, predictions))

In [ ]:
#find AUC for ROC
y_scores_gb = gb.decision_function(X_validation)
fpr_gb, tpr_gb, _ = roc_curve(y_validation, y_scores_gb)
roc_auc_gb = auc(fpr_gb, tpr_gb)

print("Area under ROC curve = {:0.2f}".format(roc_auc_gb))